# Kiss3DGen - Pipeline Image to 3D

Este notebook contém todo o código necessário para executar o pipeline Kiss3DGen no Google Colab.

## 📋 Índice
1. Instalação de Dependências
2. Configuração do Ambiente
3. Download de Modelos
4. Execução do Pipeline

---


## 1. Instalação de Dependências

Primeiro, vamos instalar todas as dependências necessárias.


In [ ]:
# Verificar Python version
import sys
print(f"Python version: {sys.version}")
print(f"Python path: {sys.executable}")


In [ ]:
# Instalar PyTorch com CUDA (Colab geralmente tem CUDA 11.8 ou 12.1)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [ ]:
# Verificar CUDA
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


In [ ]:
# Instalar dependências básicas do requirements.txt
!pip install einops==0.8.0 imageio==2.36.1 imageio-ffmpeg==0.5.1 Imath==0.0.2
!pip install jaxtyping==0.2.36 kiui==0.2.10 mathutils==3.3.0 matplotlib==3.9.2
!pip install omegaconf==2.3.0 onnxruntime==1.18.0 open3d==0.18.0
!pip install opencv-python==4.10.0.84 opencv-python-headless==4.9.0.80
!pip install OpenEXR==3.3.2 Pillow==11.0.0 plyfile==1.1 PyGLM==2.7.1
!pip install pygltflib==1.16.3 PyMCubes==0.1.4 pymeshlab==2023.12.post1
!pip install PyOpenGL==3.1.0 PyYAML==6.0.1 rembg==2.0.57 scipy==1.13.1
!pip install tqdm==4.66.4 transformers==4.47.1 trimesh==4.4.0
!pip install webdataset==0.2.86 xatlas==0.0.9 ninja
!pip install sentencepiece accelerate huggingface-hub diffusers
!pip install fvcore pytorch-lightning==2.2.0 PEFT==0.10.0 pyrender==0.1.45 timm
!pip install protobuf numpy==1.26.3


In [ ]:
# Instalar PyTorch3D
!pip install "git+https://github.com/facebookresearch/pytorch3d.git@stable"


In [ ]:
# Instalar nvdiffrast (pode demorar alguns minutos)
!pip install git+https://github.com/NVlabs/nvdiffrast


## 2. Configuração do Ambiente

Agora vamos configurar o ambiente e clonar o repositório Kiss3DGen.


In [ ]:
# Clonar repositório Kiss3DGen
import os
from pathlib import Path

# Definir caminhos
WORK_DIR = Path("/content")
KISS3D_ROOT = WORK_DIR / "Kiss3DGen"

# Clonar repositório se não existir
if not KISS3D_ROOT.exists():
    print("Clonando repositório Kiss3DGen...")
    !git clone https://github.com/EnVision-Research/Kiss3DGen.git
    print("[OK] Repositório clonado")
else:
    print(f"[OK] Repositório já existe em {KISS3D_ROOT}")

# Se você já tem os arquivos no Colab em outro local, descomente e ajuste:
# KISS3D_ROOT = Path("/content/drive/MyDrive/kiss3d_reproduction/Kiss3DGen")

print(f"Kiss3DGen root: {KISS3D_ROOT}")
print(f"Existe: {KISS3D_ROOT.exists()}")


In [ ]:
# Adicionar ao path do Python
import sys
if str(KISS3D_ROOT) not in sys.path:
    sys.path.insert(0, str(KISS3D_ROOT))
    sys.path.insert(0, str(WORK_DIR))

# Mudar para o diretório do Kiss3DGen
os.chdir(str(KISS3D_ROOT))
print(f"Working directory: {os.getcwd()}")


In [ ]:
# Instalar diffusers customizado do Kiss3DGen
custom_diffusers_path = KISS3D_ROOT / "custom_diffusers"
if custom_diffusers_path.exists():
    os.chdir(str(custom_diffusers_path))
    !pip install -e .
    os.chdir(str(KISS3D_ROOT))
    print("[OK] Diffusers customizado instalado")
else:
    print(f"[AVISO] custom_diffusers não encontrado em {custom_diffusers_path}")


In [ ]:
# Configurar variáveis de ambiente
import os
os.environ.setdefault("XFORMERS_FORCE_DISABLE_TRITON", "1")
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

print("[OK] Variáveis de ambiente configuradas")


In [ ]:
# Autenticar no HuggingFace (necessário para baixar modelos)
from huggingface_hub import login, whoami

try:
    # Verificar se já está autenticado
    user = whoami()
    print(f"[OK] Já autenticado como: {user.get('name', 'N/A')}")
except:
    # Se não estiver autenticado, pedir token
    print("Por favor, forneça seu token do HuggingFace:")
    print("1. Acesse: https://huggingface.co/settings/tokens")
    print("2. Crie um token com permissões de leitura")
    print("3. Cole o token abaixo")
    
    # Use esta alternativa para colar o token diretamente:
    from getpass import getpass
    HF_TOKEN = getpass("Cole seu token do HuggingFace: ")
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("[OK] Autenticado no HuggingFace")


## 3. Download de Modelos

Agora vamos baixar os modelos necessários do HuggingFace.


In [ ]:
# Verificar quais modelos já estão baixados
from huggingface_hub import snapshot_download
import os

REQUIRED_MODELS = {
    "zero123": {
        "repo_id": "sudo-ai/zero123plus-v1.2",
        "description": "Zero123++ para geração de multiview",
        "required": True
    },
    "flux": {
        "repo_id": "black-forest-labs/FLUX.1-dev",
        "description": "Flux diffusion model",
        "required": True
    },
    "controlnet": {
        "repo_id": "InstantX/FLUX.1-dev-Controlnet-Union",
        "description": "ControlNet para Flux",
        "required": True
    },
    "redux": {
        "repo_id": "black-forest-labs/FLUX.1-Redux-dev",
        "description": "Flux Prior Redux",
        "required": False
    }
}

def check_model_downloaded(repo_id: str) -> bool:
    """Verifica se um modelo está baixado"""
    cache_dir = os.path.expanduser("~/.cache/huggingface/hub")
    model_dir_name = f"models--{repo_id.replace('/', '--')}"
    model_path = os.path.join(cache_dir, model_dir_name)
    return os.path.exists(model_path) and len(os.listdir(model_path)) > 0

print("Verificando modelos...")
for name, info in REQUIRED_MODELS.items():
    repo_id = info["repo_id"]
    if check_model_downloaded(repo_id):
        print(f"[OK] {name}: Já baixado")
    else:
        print(f"[PENDENTE] {name}: Precisa baixar")


In [ ]:
# Baixar modelos necessários (isso pode demorar bastante tempo)
print("="*60)
print("DOWNLOAD DE MODELOS")
print("="*60)
print("\nAVISO: Isso pode demorar várias horas e requer muito espaço em disco!")
print("Modelos grandes serão baixados do HuggingFace.\n")

for name, info in REQUIRED_MODELS.items():
    repo_id = info["repo_id"]
    description = info["description"]
    required = info["required"]
    
    if check_model_downloaded(repo_id):
        print(f"\n[OK] {name} já está baixado - pulando")
        continue
    
    if not required:
        print(f"\n[AVISO] {name} é opcional - pulando")
        continue
    
    print(f"\n{'='*60}")
    print(f"Baixando: {name}")
    print(f"Repositório: {repo_id}")
    print(f"Descrição: {description}")
    print(f"{'='*60}")
    
    try:
        snapshot_download(
            repo_id=repo_id,
            local_dir=None,  # Usar cache padrão
            local_dir_use_symlinks=False,
            resume_download=True
        )
        print(f"[OK] {name} baixado com sucesso!")
    except Exception as e:
        print(f"[ERRO] Erro ao baixar {name}: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "="*60)
print("DOWNLOAD CONCLUÍDO")
print("="*60)


In [ ]:
# Baixar modelos adicionais do Kiss3DGen (LoRA, checkpoint, etc.)
from huggingface_hub import hf_hub_download
import shutil

print("Baixando modelos adicionais do Kiss3DGen...")

# Criar diretório para checkpoints
checkpoint_dir = KISS3D_ROOT / "checkpoint"
checkpoint_dir.mkdir(parents=True, exist_ok=True)

# LoRA RGB/Normal
lora_path = checkpoint_dir / "flux_lora" / "rgb_normal.safetensors"
lora_path.parent.mkdir(parents=True, exist_ok=True)
if not lora_path.exists():
    print("Baixando LoRA RGB/Normal...")
    hf_hub_download(
        repo_id="LTT/Kiss3DGen",
        filename="rgb_normal.safetensors",
        local_dir=str(lora_path.parent),
        repo_type="model"
    )
    print("[OK] LoRA baixado")
else:
    print("[OK] LoRA já existe")

# FlexGen checkpoint para Zero123++
flexgen_path = checkpoint_dir / "zero123++" / "flexgen.ckpt"
flexgen_path.parent.mkdir(parents=True, exist_ok=True)
if not flexgen_path.exists():
    print("Baixando FlexGen checkpoint...")
    hf_hub_download(
        repo_id="LTT/Kiss3DGen",
        filename="flexgen.ckpt",
        local_dir=str(flexgen_path.parent),
        repo_type="model"
    )
    print("[OK] FlexGen checkpoint baixado")
else:
    print("[OK] FlexGen checkpoint já existe")

# LRM checkpoint
lrm_path = checkpoint_dir / "lrm" / "final_ckpt.ckpt"
lrm_path.parent.mkdir(parents=True, exist_ok=True)
if not lrm_path.exists():
    print("Baixando LRM checkpoint...")
    hf_hub_download(
        repo_id="LTT/PRM",
        filename="final_ckpt.ckpt",
        local_dir=str(lrm_path.parent),
        repo_type="model"
    )
    print("[OK] LRM checkpoint baixado")
else:
    print("[OK] LRM checkpoint já existe")

# Verificar se init_3d_Bundle existe (necessário para o pipeline)
init_bundle_dir = KISS3D_ROOT / "init_3d_Bundle"
if not init_bundle_dir.exists() or len(list(init_bundle_dir.glob("*.png"))) == 0:
    print("\n[AVISO] init_3d_Bundle não encontrado ou vazio!")
    print("Este diretório contém imagens template necessárias para o pipeline.")
    print("Se o repositório foi clonado corretamente, deve estar presente.")
    print(f"Verificando: {init_bundle_dir}")
    if init_bundle_dir.exists():
        print(f"Arquivos encontrados: {list(init_bundle_dir.glob('*'))}")
else:
    print(f"[OK] init_3d_Bundle encontrado com {len(list(init_bundle_dir.glob('*.png')))} imagens")


In [ ]:
# Verificar se os modelos necessários estão presentes
print("Verificando estrutura do repositório Kiss3DGen...")

required_dirs = {
    "models/zero123plus": "Modelo Zero123++ para multiview",
    "models/lrm": "Modelo LRM para reconstrução",
    "models/ISOMER": "Modelo ISOMER para refinamento",
    "pipeline/custom_pipelines": "Pipelines customizados do Flux",
    "pipeline/pipeline_config": "Configurações do pipeline",
    "utils": "Utilitários do Kiss3DGen"
}

missing = []
for dir_name, description in required_dirs.items():
    dir_path = KISS3D_ROOT / dir_name
    if dir_path.exists():
        print(f"[OK] {dir_name}: {description}")
    else:
        print(f"[ERRO] {dir_name} NÃO ENCONTRADO: {description}")
        missing.append(dir_name)

if missing:
    print(f"\n[AVISO] Diretórios faltando: {missing}")
    print("Isso pode indicar que o repositório não foi clonado completamente.")
    print("Tente clonar novamente ou verificar se o repositório está completo.")
else:
    print("\n[OK] Todos os diretórios necessários estão presentes!")


## 4. Execução do Pipeline

Agora vamos executar o pipeline completo de Image to 3D. Primeiro, vamos importar o código necessário do repositório.


In [ ]:
# Importar o wrapper do Kiss3DGen diretamente do pipeline
# IMPORTANTE: Garantir que estamos no diretório correto
os.chdir(str(KISS3D_ROOT))

# Adicionar custom_pipelines ao path antes de importar
CUSTOM_PIPELINE_DIR = KISS3D_ROOT / "pipeline" / "custom_pipelines"
if str(CUSTOM_PIPELINE_DIR) not in sys.path:
    sys.path.insert(0, str(CUSTOM_PIPELINE_DIR))

from pipeline.kiss3d_wrapper import init_wrapper_from_config, run_image_to_3d
from pipeline.utils import logger, TMP_DIR, OUT_DIR

# Criar diretórios de saída
os.makedirs(TMP_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

print(f"[OK] Diretórios configurados:")
print(f"  Working directory: {os.getcwd()}")
print(f"  TMP_DIR: {TMP_DIR}")
print(f"  OUT_DIR: {OUT_DIR}")
print(f"  Custom pipelines: {CUSTOM_PIPELINE_DIR}")


In [ ]:
# Carregar configuração
config_path = KISS3D_ROOT / "pipeline" / "pipeline_config" / "default.yaml"

# Se não existir, criar uma configuração básica para Colab
if not config_path.exists():
    print("Criando configuração padrão para Colab...")
    config_path.parent.mkdir(parents=True, exist_ok=True)
    
    config_content = """
flux:
  base_model: "black-forest-labs/FLUX.1-dev"
  dtype: 'bf16'  # Nota: usar 'dtype' não 'flux_dtype'
  lora: "./checkpoint/flux_lora/rgb_normal.safetensors"
  controlnet: "InstantX/FLUX.1-dev-Controlnet-Union"
  redux: "black-forest-labs/FLUX.1-Redux-dev"
  num_inference_steps: 20
  seed: 42
  device: 'cuda:0'

multiview:
  base_model: "sudo-ai/zero123plus-v1.2"
  custom_pipeline: "./models/zero123plus"
  unet: "./checkpoint/zero123++/flexgen.ckpt"
  num_inference_steps: 50
  seed: 42
  device: 'cuda:0'

reconstruction:
  model_config: "./models/lrm/config/PRM_inference.yaml"
  base_model: "./checkpoint/lrm/final_ckpt.ckpt"
  device: 'cuda:0'

caption:
  base_model: "multimodalart/Florence-2-large-no-flash-attn"
  device: 'cuda:0'

llm:
  base_model: "Qwen/Qwen2-7B-Instruct"
  device: 'cpu'  # Usar CPU no Colab para economizar VRAM

use_zero_gpu: false
3d_bundle_templates: './init_3d_Bundle'
"""
    
    with open(config_path, 'w') as f:
        f.write(config_content)
    print(f"[OK] Configuração criada em {config_path}")
else:
    print(f"[OK] Usando configuração existente: {config_path}")

print(f"[OK] Configuração carregada de {config_path}")


In [ ]:
# Inicializar o wrapper (isso pode demorar vários minutos)
print("Inicializando pipeline Kiss3DGen...")
print("AVISO: Isso pode demorar vários minutos para carregar todos os modelos!")

# IMPORTANTE: A função init_wrapper_from_config original só aceita config_path
# Se você precisar de fast_mode ou outras opções, edite o config.yaml diretamente
try:
    k3d_wrapper = init_wrapper_from_config(str(config_path))
    print("\n[OK] Pipeline inicializado com sucesso!")
    
    # Verificar se os modelos foram carregados
    if hasattr(k3d_wrapper, 'flux_pipeline') and k3d_wrapper.flux_pipeline is not None:
        print("[OK] Flux pipeline carregado")
    if hasattr(k3d_wrapper, 'multiview_pipeline') and k3d_wrapper.multiview_pipeline is not None:
        print("[OK] Multiview pipeline carregado")
    if hasattr(k3d_wrapper, 'recon_model') and k3d_wrapper.recon_model is not None:
        print("[OK] Reconstruction model carregado")
        
except Exception as e:
    print(f"\n[ERRO] Falha ao inicializar pipeline: {e}")
    import traceback
    traceback.print_exc()
    print("\n[INFO] Verifique:")
    print("  1. Se todos os modelos foram baixados corretamente")
    print("  2. Se o repositório Kiss3DGen foi clonado completamente")
    print("  3. Se os caminhos no config.yaml estão corretos")
    print("  4. Se há espaço suficiente em disco e VRAM")


In [ ]:
# Preparar imagem de entrada
# Você pode fazer upload de uma imagem ou usar uma URL

from google.colab import files
from PIL import Image
import requests
from io import BytesIO

# Opção 1: Fazer upload de uma imagem
print("Opção 1: Fazer upload de uma imagem")
uploaded = files.upload()
input_image_path = list(uploaded.keys())[0] if uploaded else None

# Opção 2: Usar uma URL (descomente se preferir)
# image_url = "https://example.com/image.jpg"
# response = requests.get(image_url)
# input_image = Image.open(BytesIO(response.content))
# input_image_path = "/tmp/input_image.jpg"
# input_image.save(input_image_path)

if input_image_path:
    print(f"Imagem carregada: {input_image_path}")
    input_image = Image.open(input_image_path)
    display(input_image)
else:
    print("Nenhuma imagem carregada. Por favor, faça upload de uma imagem.")


In [ ]:
# Executar pipeline Image to 3D
import time

if 'input_image_path' in locals() and input_image_path:
    print("="*60)
    print("EXECUTANDO PIPELINE IMAGE TO 3D")
    print("="*60)
    print(f"\nInput: {input_image_path}")
    print("\nAVISO: Isso pode demorar vários minutos!")
    
    start_time = time.time()
    
    try:
        gen_save_path, recon_mesh_path = run_image_to_3d(
            k3d_wrapper,
            input_image_path,
            enable_redux=True,
            use_mv_rgb=True,
            use_controlnet=True
        )
        
        elapsed_time = time.time() - start_time
        
        print("\n" + "="*60)
        print("PIPELINE CONCLUÍDO!")
        print("="*60)
        print(f"Tempo total: {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)")
        print(f"\nBundle image: {gen_save_path}")
        print(f"Mesh 3D: {recon_mesh_path}")
        
        # Mostrar resultados
        if os.path.exists(gen_save_path):
            bundle_img = Image.open(gen_save_path)
            display(bundle_img)
        
        print(f"\nMesh 3D salvo em: {recon_mesh_path}")
        
    except Exception as e:
        print(f"\n[ERRO] Falha ao executar pipeline: {e}")
        import traceback
        traceback.print_exc()
else:
    print("Por favor, carregue uma imagem primeiro.")


In [ ]:
# Download dos resultados (opcional)
if 'recon_mesh_path' in locals() and os.path.exists(recon_mesh_path):
    print(f"Baixando mesh 3D: {recon_mesh_path}")
    files.download(recon_mesh_path)
    
if 'gen_save_path' in locals() and os.path.exists(gen_save_path):
    print(f"Baixando bundle image: {gen_save_path}")
    files.download(gen_save_path)

print("\n[OK] Downloads concluídos!")
